# MODEL TRAINING

In [52]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [53]:
data_train = pd.read_csv('../Data/train.csv')
data_test = pd.read_csv('../Data/test.csv')

In [54]:
X, y = data_train.drop(columns="SalePrice"), data_train["SalePrice"].copy()
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=42)
X_test = data_test.copy()

In [55]:
preprocessor = pickle.load(open('./preprocessor.pkl', 'rb'))

In [56]:
model = LinearRegression()

In [57]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model) 
])

In [58]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF...
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu

In [59]:
with open('./classifier.pkl', 'wb') as pkl_file:
    pickle.dump(clf, pkl_file)